In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-rimbwy29f75muq"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [2]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node, custom_metric_map

tf_meta_prompt, test_cases = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 

starter_code_str = """def generate_prompt(project_description):
    return (
        f"Given the grant application description: ### Description ### \\n{project_description}\\n ### End of Description ###, do evaluate the project and make a decision to reject it. Make sure the output is a json string in markdown like this {{\'decision\':<boolean>, \'comment\':\'<string>\'}} near the top of your response or people will die."
    )""" # add initial code string
    
node = EvolNode(tf_meta_prompt, starter_code_str, "Basic Prompt Function", get_response=get_endpoint_response, test_cases=test_cases[:5]) # setting manual test cases

In [8]:
node_dict = {"reasoning": "Basic Prompt Function", "code": starter_code_str, "fitness": 0.33}
# evolve_prompt =node._get_evolve_prompt("e2", feedback="reject all cases", parents=[node_dict])
# print(evolve_prompt)

node.evolve("e2", feedback="reject all cases", parents=[node_dict]) # Evaluation on test case completely failing here .... 

# Debug on evaluation bit


In [9]:
self = node 
from methods.evolnode import check_alignment_parallel

codes = [starter_code_str]
test_cases = test_cases[:2]
num_runs = 2
timeout = 5
max_tries = 3
test_inputs = [case[0] for case in test_cases]

if self.meta_prompt.mode == PromptMode.CODE: 
    output_per_code_per_test, errors_per_code_per_test = self.call_code_function_parallel(test_inputs, codes, timeout=timeout)
elif self.meta_prompt.mode == PromptMode.PROMPT:
    output_per_code_per_test, errors_per_code_per_test = self.call_prompt_function_parallel(test_inputs, codes, max_tries)
else:
    raise ValueError(f"Unknown mode: {self.meta_prompt.mode}")

# Print info about outputs and errors per code
for code_index in output_per_code_per_test:
    print(f"\nCode {code_index} outputs:")
    for test_index in output_per_code_per_test[code_index]:
        print(f"Test {test_index}: {output_per_code_per_test[code_index][test_index]}")
        if errors_per_code_per_test[code_index][test_index]:
            print(f"Errors: {errors_per_code_per_test[code_index][test_index]}")

# alignment checking
test_inputs = [case[0] for case in test_cases]
target_outputs = [case[1] for case in test_cases]
score_per_code_per_test, evaluate_errors_per_code_per_test = check_alignment_parallel(output_per_code_per_test, test_inputs, target_outputs, 
                                                                                        self.get_response, batch_size=num_runs, custom_metric_map=custom_metric_map) # not sure if buggy or not


Concurrent Execution of 2 Node functions to generate prompts ...: 100%|██████████| 2/2 [00:00<00:00, 25343.23it/s]
Processing LLM queries: 100%|██████████| 6/6 [00:10<00:00,  1.73s/it]


 :: Total time elapsed: 10.38s, 0 errors

Code 0 outputs:
Test 0: {'decision': 'No', 'comment': "The project has several critical issues, including a lack of clear objectives, proof of concept, and proposed solution. Furthermore, the grant amount sought is listed as none, which raises questions about the project's feasibility and scalability. Additionally, the project's scope of work is undefined, making it difficult to assess its viability and potential impact."}
Test 1: {'decision': 'No', 'comment': "The project, 'Marine Battery As A Service,' lacks a clear proof of concept and is in the ideation stage, which increases the risk of project failure. Additionally, the objectives and scope of work are not defined, making it challenging to assess the project's viability and potential impact. Furthermore, the absence of a specific grant amount sought suggests that the project may not be well-prepared for funding. While the project has engaged with key stakeholders, the lack of a clear plan

Processing LLM queries: 100%|██████████| 4/4 [00:16<00:00,  4.18s/it]

 :: Total time elapsed: 16.72s, 0 errors


In [11]:
# output_per_code_per_test
score_per_code_per_test

defaultdict(<function methods.meta_execute.combine_scores.<locals>.<lambda>()>,
            {0: defaultdict(float, {0: 1.0, 1: 1.0})})